In [ ]:
%pip install expelliarmus
%pip install aedat
%pip install loris
%pip install brotli
%pip install h5py
%pip install numpy

from expelliarmus import Wizard
import aedat
import pathlib
import h5py
import numpy as np
import timeit
import requests
import pickle
import os
import loris
import brotli

In [ ]:
# fname = "driving_sample"
fname = "construction" # use this one if you want to include aedat benchmarks

# key is the name of the encoding, value is the file name ending
extension_map = {
    'aedat': '.aedat4',
    'dat': '.dat',
    'evt2': '_evt2.raw',
    'evt3': '_evt3.raw',
    'hdf5': '.hdf5',
    'hdf5_lzf': '_lzf.hdf5',
    'hdf5_gzip': '_gzip.hdf5',
    'numpy': '.npy',
    'eventstream': '.es',
    'brotli': '.bin.br'
}
get_fpath = lambda encoding: f"data/file-benchmark/{fname}{extension_map[encoding]}"

In [ ]:
if fname == "driving_sample":
    if not os.path.exists(get_fpath('evt3')):
        print("Downloading EVT3 file... ", end="")
        if not pathlib.Path(get_fpath('evt3')).is_file():
            r = requests.get("https://dataset.prophesee.ai/index.php/s/nVcLLdWAnNzrmII/download", allow_redirects=True)
            open(get_fpath('evt3'), 'wb').write(r.content)
        print("done!")

    wizard = Wizard(encoding="evt3")
    data = wizard.read(get_fpath("evt3"))


if fname == "construction":
    if not os.path.exists(get_fpath('aedat')):
        print("Downloading aedat4 file... ", end="")
        if not pathlib.Path(get_fpath('aedat')).is_file():
            r = requests.get("https://cloudstor.aarnet.edu.au/plus/s/ORQ2oOz9NfwiHLZ/download?path=%2F&files=construction.aedat4", allow_redirects=True)
            open(get_fpath('aedat'), 'wb').write(r.content)
    if not os.path.exists(get_fpath('eventstream')):
        print("Downloading eventstream file... ", end="")
        if not pathlib.Path(get_fpath('eventstream')).is_file():
            r = requests.get("https://cloudstor.aarnet.edu.au/plus/s/ORQ2oOz9NfwiHLZ/download?path=%2F&files=construction.es", allow_redirects=True)
            open(get_fpath('eventstream'), 'wb').write(r.content)
        print("done!")

    decoder = aedat.Decoder(get_fpath('aedat'))
    events = np.concatenate([packet["events"] for packet in decoder if "events" in packet])
    data = events.astype(np.dtype([('t', '<i8'), ('x', '<i2'), ('y', '<i2'), ('p', 'u1')], align=True))

In [ ]:
## Generate all comparison files

# evt2 and dat
raw_encodings = ["dat", "evt2", "evt3"]
for encoding in raw_encodings:
    if not os.path.exists(get_fpath(encoding)):
        print(f"Generating file for {encoding} encoding.")
        wizard = Wizard(encoding=encoding)
        wizard.save(fpath=get_fpath(encoding), arr=data)

# variants of hdf5
hdf5_encodings = ["hdf5", "hdf5_lzf", "hdf5_gzip"]
for encoding in hdf5_encodings:
    fpath = get_fpath(encoding)
    if not os.path.exists(fpath):
        print(f"Generating file for {encoding} encoding.")
        fp = h5py.File(fpath, "w")
        fpath = pathlib.Path(get_fpath(encoding))
        if encoding=="hdf5":
            fp.create_dataset(name="events", shape=data.shape, dtype=data.dtype, data=data)
        elif encoding=="hdf5_lzf":
            fp.create_dataset(name="events", shape=data.shape, dtype=data.dtype, data=data, compression="lzf")
        elif encoding=="hdf5_gzip":
            fp.create_dataset(name="events", shape=data.shape, dtype=data.dtype, data=data, compression="gzip")
        fp.close()

# numpy
fpath = get_fpath('numpy')
if not os.path.exists(fpath):
    print(f"Generating file for numpy encoding.")
    np.save(fpath, data, allow_pickle=True)

# brotli
if not os.path.exists(get_fpath('brotli')):
    print(f"Generating file for brotli encoding.")
    with open(get_fpath('brotli'), 'wb') as out_file:
        with open(get_fpath('dat'), 'rb') as in_file:
            for i in range(3):
                l = in_file.readline()
            out_file.write(brotli.compress(in_file.read(), quality=5))

In [ ]:
## Run benchmarks

REPEAT = 1
get_fsize_MB = lambda fpath: round(fpath.stat().st_size/(1024*1024))

# evt2, evt3, dat
raw_times = []
raw_sizes = []
for encoding in raw_encodings:
    fpath = get_fpath(encoding)
    wizard = Wizard(encoding)
    wizard.set_file(fpath)
    raw_times.append(sum(timeit.repeat(lambda: wizard.read(fpath), number=1, repeat=REPEAT))/REPEAT)
    raw_sizes.append(get_fsize_MB(pathlib.Path(fpath)))

# hdf5 variants
hdf5_times = []
hdf5_sizes = []
for encoding in hdf5_encodings:
    fpath = get_fpath(encoding)
    fp = h5py.File(fpath)
    hdf5_times.append(sum(timeit.repeat(lambda: fp["events"][:], number=1, repeat=REPEAT))/REPEAT)
    fp.close()
    hdf5_sizes.append(get_fsize_MB(pathlib.Path(fpath)))

# numpy
fpath = get_fpath('numpy')
numpy_time = sum(timeit.repeat(lambda: np.load(fpath), number=1, repeat=REPEAT))/REPEAT
numpy_size = get_fsize_MB(pathlib.Path(fpath))

# aedat4
def load_aedat(fpath):
    decoder = aedat.Decoder(fpath)
    events = [packet["events"] for packet in decoder if "events" in packet]

fpath = get_fpath('aedat')
aedat_time = sum(timeit.repeat(lambda: load_aedat(fpath), number=1, repeat=REPEAT))/REPEAT
aedat_size = get_fsize_MB(pathlib.Path(fpath))

# eventstream
fpath = get_fpath('eventstream')
es_time = sum(timeit.repeat(lambda: loris.read_file(fpath), number=1, repeat=REPEAT))/REPEAT
es_size = get_fsize_MB(pathlib.Path(fpath))

# brotli
fpath = get_fpath('brotli')
dtype = np.dtype([('t', '<i8'), ('x', '<i2'), ('y', '<i2'), ('p', 'u1')])
                 
def brotli_read(): 
    # Reading to a numpy buffer the data.
    with open(fpath, "rb") as fp:
        np_buff = np.frombuffer(brotli.decompress(fp.read()), dtype=np.uint64, align=True)

    # Creating the structured NumPy array.
    arr = np.empty(len(np_buff), dtype=dtype)

    # Decoding the buffer.
    arr['t'] = np_buff & 0xFFFFFFFF # 32 bits
    arr['x'] = (np_buff >> (64-32)) & 0x3FFF # 14 bits
    arr['y'] = (np_buff >> (64-(32+14))) & 0x3FFF # 14 bits
    arr['p'] = (np_buff >> (64-(32+28)))
    return arr 

brotli_time = sum(timeit.repeat(brotli_read, number=1, repeat=REPEAT))/REPEAT
brotli_size = get_fsize_MB(pathlib.Path(fpath))

In [ ]:
## Aggregate results

import pandas as pd

df = pd.DataFrame({
    'Encoding': raw_encodings + hdf5_encodings + ["numpy", "aedat4", "eventstream", "dat/brotli"],
    'Framework': ["expelliarmus"] * len(raw_encodings) + ["h5py"] * len(hdf5_encodings) + ["numpy", "aedat", "loris", "expelliarmus/brotli"],
    'Read time [s]': raw_times + hdf5_times + [numpy_time, aedat_time, es_time, brotli_time] ,
    'File size [MB]': raw_sizes + hdf5_sizes + [numpy_size, aedat_size, es_size, brotli_size],
})

In [ ]:
## Plot results

import plotly.express as px
from IPython.display import Image

title = f"Reading the same {int(len(data)/1e6)} million events from different files."
fig = px.scatter(df, x='Read time [s]', y='File size [MB]', color='Framework', symbol='Encoding', title=title, template="plotly_dark",)
fig.update_traces(marker_size=13)
fig.write_image('file_read_benchmark.png')